In [1]:
import sys
import time
import random
from configparser import ConfigParser

# config = ConfigParser()
# config.read("config.ini")
# CARLA_PATH = config.get("main", "CARLA_PATH")
# # Enable import of 'carla'
# sys.path.append(CARLA_PATH + "PythonAPI/carla/dist/carla-0.9.9-py3.6-linux-x86_64.egg")

import carla

from topology_extraction import get_opendrive_tree, get_junction_topology, get_junction_roads_topology

sys.path.append("../../../global_route_planner/")
from global_planner import get_client, draw_waypoints, spawn_vehicle

#### Initialize the CARLA Interface

In [26]:
# Get client to interact with CARLA server
client = get_client()

# Get current CARLA world
world = client.get_world()

Connection to CARLA server established!


#### Get all waypoints + define a custom draw function to visualize points on the CARLA world. 

In [27]:
all_waypoints = world.get_map().generate_waypoints(1)
def draw(road_id, lane_id=None, color=[0,255,0]):
    for wp in all_waypoints:
        
        if lane_id == None and wp.road_id == road_id:
            draw_waypoints(world, [wp], 3, color) 
#             print(wp.lane_id)
        elif wp.road_id == road_id and wp.lane_id==lane_id:
            draw_waypoints(world, [wp], 3, color) 
            
def filter_waypoints(waypoints, road_id, lane_id = None):
    
    filtered_waypoints = []
    for wp in waypoints:
        
        if lane_id == None and wp.road_id == road_id:
            filtered_waypoints.append(wp) 
        elif wp.road_id == road_id and wp.lane_id==lane_id:
            filtered_waypoints.append(wp) 

    return filtered_waypoints

def getXYZ(waypoint):
    
    return [waypoint.transform.location.x, waypoint.transform.location.y, waypoint.transform.location.z]

#### Get the junction and road topologies

In [28]:
tree = get_opendrive_tree(world)
junction_topology = get_junction_topology(tree)
road_topology = get_junction_roads_topology(tree)

In [29]:
junction_topology.keys()

dict_keys([224, 965, 421, 1175, 905, 1162, 139, 1260, 685, 334, 751, 1148, 1050, 53, 599, 1070, 943, 509, 924, 829, 245])

#### Visualize intersection paths for any junction (Uncomment to Visualize)

In [33]:
junction_id = 1050  ## Pick from junction_topology.keys()

for idx in range(len(junction_topology[junction_id])):
    intersection_road_id, intersection_lane_id = junction_topology[junction_id][idx][1]

    road_1_id, road_2_id, lane_connections = road_topology[intersection_road_id]
    print(road_1_id, road_2_id, intersection_road_id, lane_connections)
    
    for intersection_connection_lanes in lane_connections:
                
        direction = intersection_connection_lanes[0]
        lane_1_id = intersection_connection_lanes[1]
        lane_2_id = intersection_connection_lanes[-1]

        if(direction == "forward"):
            draw(road_1_id,lane_1_id,color=[0,255,0])
            draw(road_2_id,lane_2_id,color=[255,0,0])
        else:
            draw(road_1_id,lane_1_id,color=[255,0,0])
            draw(road_2_id,lane_2_id,color=[0,255,0])            
        
        used_lane_IDs = []
        for i in range(2,len(intersection_connection_lanes)-1):
            if intersection_connection_lanes[i] in used_lane_IDs:
                continue
            used_lane_IDs.append(intersection_connection_lanes[i])
            draw(intersection_road_id, intersection_connection_lanes[i],color=[0,0,255])


41 42 1058 [['backward', 2, 2, 2, 2, 2], ['backward', 1, 1, 1, 1, 1]]
41 42 1058 [['backward', 2, 2, 2, 2, 2], ['backward', 1, 1, 1, 1, 1]]
41 42 1059 [['forward', -1, -1, -1, -1, -1], ['forward', -2, -2, -2, -2, -2]]
41 42 1059 [['forward', -1, -1, -1, -1, -1], ['forward', -2, -2, -2, -2, -2]]
41 51 1060 [['backward', 1, 1, 1, 1, 1]]
41 51 1063 [['forward', -2, -1, -2]]
42 51 1064 [['forward', 1, -1, -1, -1]]
42 51 1066 [['backward', -2, 1, 2, 2, 2, 2]]


#### Collect all paths for a junction

In [43]:
junction_id = 53  ## Pick from junction_topology.keys()

junction_data = []
for idx in range(len(junction_topology[junction_id])):
    intersection_road_id, intersection_lane_id = junction_topology[junction_id][idx][1]
    incoming_road_id, incoming_lane_id = junction_topology[junction_id][idx][0]
    contactPoint = junction_topology[junction_id][idx][2]
    
    
    print(idx)
    if(contactPoint == "start"):
        print("Road", incoming_road_id, "goes into connecting road")
        print("Road", incoming_road_id, "is green")

    else:
        print("Road", incoming_road_id, "is going out of connecting road")
        print("Road", incoming_road_id, "is red")

    
    road_1_id, road_2_id, lane_connections = road_topology[intersection_road_id]
    
#     print("Road", road_1_id, "is green")
#     print("Road", road_2_id, "is red")
    print("\n")

    for intersection_connection_lanes in lane_connections:
                
        lane_1_id = intersection_connection_lanes[0]
        lane_2_id = intersection_connection_lanes[-1]

        lane_1_waypoints = filter_waypoints(all_waypoints, road_1_id,lane_1_id)
        lane_2_waypoints = filter_waypoints(all_waypoints, road_2_id,lane_2_id)
        
        connection_waypoints = []
        used_lane_IDs = []
        for i in range(1,len(intersection_connection_lanes)-1):
            if intersection_connection_lanes[i] in used_lane_IDs:
                continue
            used_lane_IDs.append(intersection_connection_lanes[i])
            connection_waypoints += filter_waypoints(all_waypoints, intersection_road_id, intersection_connection_lanes[i])
    
        junction_data.append([lane_1_waypoints, lane_2_waypoints, connection_waypoints])


0
Road 5 is going out of connecting road
Road 5 is red


1
Road 5 is going out of connecting road
Road 5 is red


2
Road 4 goes into connecting road
Road 4 is green


3
Road 4 goes into connecting road
Road 4 is green


4
Road 4 goes into connecting road
Road 4 is green


5
Road 7 is going out of connecting road
Road 7 is red


6
Road 8 is going out of connecting road
Road 8 is red


7
Road 4 goes into connecting road
Road 4 is green


8
Road 7 is going out of connecting road
Road 7 is red


9
Road 5 goes into connecting road
Road 5 is green


10
Road 5 goes into connecting road
Road 5 is green


11
Road 8 is going out of connecting road
Road 8 is red


12
Road 8 is going out of connecting road
Road 8 is red


13
Road 8 is going out of connecting road
Road 8 is red


14
Road 7 goes into connecting road
Road 7 is green


15
Road 7 goes into connecting road
Road 7 is green




#### Select one of the paths

In [46]:
idx = 3

entry_path = junction_data[idx][0]
exit_path = junction_data[idx][1]
connection_path = junction_data[idx][2]


In [50]:
draw_waypoints(world,connection_path[:5],3)

#### Spawn vehicles on the entry and exit paths

In [21]:
if 'ego' in locals():
    ego.destroy()
if 'actor' in locals():
    actor.destroy()
    
ego_spawn_transform = random.choice(entry_path).transform
actor_spawn_transform = random.choice(exit_path).transform

ego_location = carla.Location(x = ego_spawn_transform.location.x, y=ego_spawn_transform.location.y, z=ego_spawn_transform.location.z + 1)
ego_rotation = carla.Rotation(yaw = ego_spawn_transform.rotation.yaw)
ego_spawn_transform = carla.Transform(location=ego_location, rotation=ego_rotation)
ego = spawn_vehicle(world, "model3", spawn_point=ego_spawn_transform)

actor_location = carla.Location(x = actor_spawn_transform.location.x, y=actor_spawn_transform.location.y, z=actor_spawn_transform.location.z + 1)
actor_rotation = carla.Rotation(yaw = actor_spawn_transform.rotation.yaw)
actor_spawn_transform = carla.Transform(location=actor_location, rotation=actor_rotation)
actor = spawn_vehicle(world, "model3", spawn_point=actor_spawn_transform)

draw_waypoints(world, entry_path, 5, color = [0,255,0])
draw_waypoints(world, exit_path, 5, color = [255,0,0])
draw_waypoints(world, connection_path, 5, color = [0,0,255])


#### Get intersection point using start and end waypoints of connection path

In [ ]:
x1, y1, z1 = getXYZ(connection_path[0])
x2, y2, z2 = getXYZ(connection_path[1])
x3, y3, z3 = getXYZ(connection_path[-1])
x4, y4, z4 = getXYZ(connection_path[-2])

intersection_x = ((x1*y2 - y1*x2)*(x3-x4) - (x1-x2)*(x3*y4 - y3*x4)) / ((x1-x2)*(y3-y4) - (y1-y2)*(x3-x4))
intersection_y = ((x1*y2 - y1*x2)*(y3-y4) - (y1-y2)*(x3*y4 - y3*x4)) / ((x1-x2)*(y3-y4) - (y1-y2)*(x3-x4))

intersection_location = carla.Location(x=intersection_x, y=intersection_y, z=(z1+z2+z3+z4)/4)

world.debug.draw_string(intersection_location, "0", draw_shadow=False, color=carla.Color(r=0,g=255,b=0), life_time=10)


In [ ]:

draw_waypoints(world, entry_path[-50:], 3, color = [0,255,0])
draw_waypoints(world, exit_path[:50], 3, color = [255,0,0])
world.debug.draw_string(intersection_location, "0", draw_shadow=False, color=carla.Color(r=0,g=50,b=160), life_time=10)
